In [ ]:
! pip install dotenv

In [ ]:
import pandas as pd
df= pd.read_excel('/content/data.xlsx')
df

,facebookUrl,listing_price/amount,listing_price/formatted_amount,location/reverse_geocode/city,text
0,https://www.facebook.com/marketplace/algiers/s...,100.0,DZD100,الجزائر,Vend voiture
1,https://www.facebook.com/marketplace/algiers/s...,2.0,DZD2,الجزائر,ved voirure
2,https://www.facebook.com/marketplace/algiers/s...,137.0,DZD137,الجزائر,Zotye nomad 2
3,https://www.facebook.com/marketplace/algiers/s...,123.0,DZD123,الجزائر,206 ess 1.6 16v
4,https://www.facebook.com/marketplace/algiers/s...,1.0,DZD1,رغاية,Nissan tiida boite auto 2008 tres tres propre
...,...,...,...,...,...
557,https://www.facebook.com/reel/616206774590874/,NaN,NaN,NaN,و يقولك عطاوني 100 ماشي بعيدة \n\n#fromourhood...
558,https://www.facebook.com/reel/1290333358928421/,NaN,NaN,NaN,نكذبوا اوو؟\n\n#fromourhooddz #ramadan #ramada...
559,https://www.facebook.com/reel/537925589322425/,NaN,NaN,NaN,رمضان يجمعنا❤\nدير جام للصفحة ❤\n\n#fromourhoo...
560,https://www.facebook.com/permalink.php?story_f...,NaN,NaN,NaN,تم حذر صفحتنا الأولى للأسف\nدير جام و دعما في ...


In [ ]:
import pandas as pd
import openai
import time
import json
import tiktoken
import re
import concurrent.futures
import os

# Setup system prompt for car listings
SYSTEM_PROMPT = """
You are a car listing assistant specialized in Algerian and French vehicle descriptions.

Your task is to extract structured information from short, unstructured car listings.
Use both the content of the text **and your knowledge of automotive specifications** to fill in missing details, such as brand, engine size, fuel type, and transmission.

If a value like engine, fuel, brand, or transmission is not explicitly mentioned, but it can be reasonably inferred from the model and year — **fill it in using your knowledge**.

If the number of kilometers (`km`) or price are not mentioned in the text, set them to `null`. Do not guess them.

Return only a JSON array of results, structured as follows:
- brand
- model
- year
- engine
- fuel ("essence" or "diesel")
- km (numeric, null if not mentioned)
- price (numeric, null if not mentioned)
- boite ("automatic", "manual", or null)
- condition (e.g., "very clean", "good", or null)
- transaction ("sell" or "rent")

### Examples:

```json
[
    {
        "input": "208 essence 2015 1.2 160000 km",
        "output": [
            {
                "brand": "Peugeot",
                "model": "208",
                "year": 2015,
                "engine": "1.2",
                "fuel": "essence",
                "km": 160000,
                "price": null,
                "boite": "manual",
                "condition": null,
                "transaction": "sell"
            }
        ]
    },
    {
        "input": "Nissan tiida boite auto 2008 tres tres propre",
        "output": [
            {
                "brand": "Nissan",
                "model": "Tiida",
                "year": 2008,
                "engine": "1.6",
                "fuel": "essence",
                "km": null,
                "price": null,
                "boite": "automatic",
                "condition": "very clean",
                "transaction": "sell"
            }
        ]
    },
    {
        "input": "Ford fiesta 2011 ess 1.25",
        "output": [
            {
                "brand": "Ford",
                "model": "Fiesta",
                "year": 2011,
                "engine": "1.25",
                "fuel": "essence",
                "km": null,
                "price": null,
                "boite": "manual",
                "condition": null,
                "transaction": "sell"
            }
        ]
    }
]
"""

# Tokenizer setup
tokenizer = tiktoken.get_encoding("cl100k_base")
MAX_TOKENS = 62000

def estimate_tokens(text):
    """Estimate token count for a given text"""
    return len(tokenizer.encode(str(text)))

def create_batches(text_list, max_tokens=MAX_TOKENS):
    """Create batches of texts that fit within token limits"""
    batches = []
    current_batch = []
    current_tokens = 0

    system_tokens = estimate_tokens(SYSTEM_PROMPT)
    # Reserve tokens for system prompt, response, and safety buffer
    available_tokens = max_tokens - system_tokens - 5000

    for i, text in enumerate(text_list):
        tokens = estimate_tokens(text)

        # If adding this listing exceeds max tokens or batch size limit, start a new batch
        if current_tokens + tokens > available_tokens or len(current_batch) >= 30:
            if current_batch:  # Only append if batch is not empty
                batches.append(current_batch)
            current_batch = []
            current_tokens = 0

        # Add text to current batch
        current_batch.append({"id": i, "text": str(text)})
        current_tokens += tokens

    # Add the last batch if not empty
    if current_batch:
        batches.append(current_batch)

    return batches

def repair_json(json_str):
    """Repair potentially truncated JSON strings"""
    json_str = json_str.strip()

    # If it doesn't end with ']', try to fix it
    if not json_str.endswith(']'):
        try:
            # Try to parse as is first
            json.loads(json_str)
            return json_str
        except json.JSONDecodeError:
            # Add missing closing bracket
            if not json_str.endswith('}'):
                json_str += '}'
            if not json_str.endswith(']'):
                json_str += ']'

    return json_str

def extract_car_data(raw_response):
    """Extract car data from API response"""
    try:
        # First try to parse the JSON directly
        repaired_json = repair_json(raw_response)
        data = json.loads(repaired_json)

        # Ensure it's a list
        if isinstance(data, dict):
            data = [data]
        elif not isinstance(data, list):
            return []

        return data

    except json.JSONDecodeError:
        # If JSON parsing fails, try to extract individual objects
        results = []
        # Look for JSON objects with car fields
        pattern = r'\{[^{}]*"brand"[^{}]*\}'
        matches = re.findall(pattern, raw_response, re.DOTALL)

        for match in matches:
            try:
                obj = json.loads(match)
                results.append(obj)
            except json.JSONDecodeError:
                continue

        return results

def process_batch_with_retry(client, batch, max_retries=3):
    """Process a batch of car listings with retry logic"""
    for attempt in range(max_retries):
        try:
            # Format the batch for processing
            prompt_text = "Extract car information from the following listings:\n\n"
            for i, item in enumerate(batch):
                prompt_text += f"{i+1}. {item['text']}\n"

            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": prompt_text}
            ]

            response = client.chat.completions.create(
                model="deepseek-reasoner",
                messages=messages,
                temperature=0.1,
                max_tokens=8000
            )

            # Extract data from the response
            raw_response = response.choices[0].message.content
            results = extract_car_data(raw_response)

            # Match results with original indices
            output = []
            for i, item in enumerate(batch):
                if i < len(results):
                    result = results[i]
                    output.append({
                        "id": item["id"],
                        "brand": result.get("brand"),
                        "model": result.get("model"),
                        "year": result.get("year"),
                        "engine": result.get("engine"),
                        "fuel": result.get("fuel"),
                        "km": result.get("km"),
                        "price": result.get("price"),
                        "boite": result.get("boite"),
                        "condition": result.get("condition"),
                        "transaction": result.get("transaction")
                    })
                else:
                    # If we don't have enough results, use null values
                    output.append({
                        "id": item["id"],
                        "brand": None,
                        "model": None,
                        "year": None,
                        "engine": None,
                        "fuel": None,
                        "km": None,
                        "price": None,
                        "boite": None,
                        "condition": None,
                        "transaction": None
                    })

            return output

        except Exception as e:
            print(f"❌ Attempt {attempt + 1} failed: {str(e)}")
            if attempt == max_retries - 1:
                # Return error placeholders for all items in batch
                return [{
                    "id": item["id"],
                    "brand": None,
                    "model": None,
                    "year": None,
                    "engine": None,
                    "fuel": None,
                    "km": None,
                    "price": None,
                    "boite": None,
                    "condition": "Error",
                    "transaction": "Error"
                } for item in batch]

            # Exponential backoff before retry
            time.sleep(2 ** attempt)

def main():
    """Main function to process car listings"""
    # Configuration
    input_file = "/content/data.xlsx"
    output_file = "extracted_car_data.json"
    excel_output = "extracted_car_data.xlsx"

    try:
        # Read input data
        print("📖 Reading input data...")
        df = pd.read_excel(input_file)
        print(f"Loaded {len(df)} rows from {input_file}")

        # Find the text column containing car listings
        if 'text' in df.columns:
            text_column = 'text'
        elif 'description' in df.columns:
            text_column = 'description'
        else:
            # Use the first text column
            text_columns = df.select_dtypes(include=['object']).columns
            if len(text_columns) > 0:
                text_column = text_columns[0]
                print(f"Using column '{text_column}' for car listings")
            else:
                raise ValueError("No text column found in the dataset")

        texts = df[text_column].astype(str).tolist()
        print(f"Found {len(texts)} car listings to process")

        # Setup DeepSeek API client
        client = openai.OpenAI(
            api_key="Your_API_key",  # Replace with your actual API key
            base_url="https://api.deepseek.com"
        )

        # Create processing batches
        batches = create_batches(texts)
        print(f"📦 Created {len(batches)} batches for processing")

        # Configure parallel processing
        max_workers = min(10, len(batches))  # Conservative worker count
        print(f"🔄 Using {max_workers} parallel workers")

        # Process batches in parallel
        all_results = []
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all batch processing tasks
            futures = [
                executor.submit(process_batch_with_retry, client, batch)
                for batch in batches
            ]

            # Collect results as they complete
            for i, future in enumerate(concurrent.futures.as_completed(futures)):
                try:
                    results = future.result()
                    all_results.extend(results)
                    print(f"✅ Completed batch {i+1}/{len(batches)}")
                except Exception as e:
                    print(f"❌ Batch {i+1} failed: {e}")

        # Sort results to maintain original order
        all_results.sort(key=lambda x: x["id"])
        print(f"📋 Processed {len(all_results)} total entries")

        # Create model-format output (input-output pairs)
        model_output = []
        for result in all_results:
            index = result["id"]
            if index < len(texts):
                model_entry = {
                    "input": texts[index],
                    "output": [{
                        "brand": result["brand"],
                        "model": result["model"],
                        "year": result["year"],
                        "engine": result["engine"],
                        "fuel": result["fuel"],
                        "km": result["km"],
                        "price": result["price"],
                        "boite": result["boite"],
                        "condition": result["condition"],
                        "transaction": result["transaction"]
                    }]
                }
                model_output.append(model_entry)

        # Save JSON output
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(model_output, f, indent=2, ensure_ascii=False)
        print(f"💾 Model-format results saved to: {output_file}")

        # Save Excel output for convenience
        car_fields = ["brand", "model", "year", "engine", "fuel", "km", "price", "boite", "condition", "transaction"]

        # Initialize new columns
        for field in car_fields:
            df[field] = None

        # Populate extracted data
        for result in all_results:
            index = result["id"]
            if index < len(df):
                for field in car_fields:
                    df.loc[index, field] = result[field]

        df.to_excel(excel_output, index=False)
        print(f"📊 Excel results saved to: {excel_output}")

        # Summary statistics
        successful_extractions = sum(1 for r in all_results if r.get("brand") is not None)
        error_count = sum(1 for r in all_results if r.get("condition") == "Error")

        print(f"\n📈 Processing Summary:")
        print(f"Total entries: {len(model_output)}")
        print(f"Successful extractions: {successful_extractions}")
        print(f"Errors: {error_count}")
        print(f"Success rate: {(successful_extractions/len(all_results)*100):.1f}%")

    except Exception as e:
        print(f"❌ Error in main execution: {e}")
        raise

if __name__ == "__main__":
    main()

📖 Reading input data...
Loaded 562 rows from /content/data.xlsx
Found 562 car listings to process
📦 Created 19 batches for processing
🔄 Using 10 parallel workers
✅ Completed batch 1/19
✅ Completed batch 2/19
✅ Completed batch 3/19
✅ Completed batch 4/19
✅ Completed batch 5/19
✅ Completed batch 6/19
✅ Completed batch 7/19
✅ Completed batch 8/19
✅ Completed batch 9/19
✅ Completed batch 10/19
✅ Completed batch 11/19
✅ Completed batch 12/19
✅ Completed batch 13/19
✅ Completed batch 14/19
✅ Completed batch 15/19
✅ Completed batch 16/19
✅ Completed batch 17/19
✅ Completed batch 18/19
✅ Completed batch 19/19
📋 Processed 562 total entries
💾 Model-format results saved to: extracted_car_data.json
📊 Excel results saved to: extracted_car_data.xlsx

📈 Processing Summary:
Total entries: 562
Successful extractions: 319
Errors: 0
Success rate: 56.8%
